## Install Dependencies

In [12]:
!pip install -U "transformers==4.40.0" --upgrade
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install accelerate


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.org/simple/


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import transformers
import torch
import json

## Inialize the models

Llama 3

In [ ]:
model_id_llama3 = "unsloth/llama-3-8b-Instruct-bnb-4bit"

llama3 = transformers.pipeline(
    "text-generation",
    model=model_id_llama3,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)

Gemma

In [ ]:
model_id_gemma = "google/gemma-7b-it"

gemma = transformers.pipeline(
    "text-generation",
    model=model_id_gemma,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)

In [ ]:
model_id_mistral = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit"

mistral = transformers.pipeline(
    "text-generation",
    model=model_id_mistral,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)

## Fallacies

In [ ]:
def gkp_prompt(input_text, pipeline):
    messages = [
            {"role"   : "user",
            "content":
            f'First generate an explanation of the following fallacies: \
            Fallacy of Credibility: Abusive Ad Hominem; Ad Populum; Appeal to False Authority; \
            Appeal to Nature; Appeal to Tradition; Guilt by Association; Tu Quoque. \
            Fallacy of Logic: Causal Oversimplification; Circular Reasoning; Equivocation; \
            False Analogy; False Causality; False Dilemma; Hasty Generalization; Slippery Slope; Straw Man; Fallacy of Division. \
            Appeal to Emotion: Appeal to Positive Emotion; Appeal to Anger; Appeal to Fear; \
            Appeal to Pity; Appeal to Ridicule; Appeal to Worse Problem. \
            Second identify which of the fallacies you just explained, occur in the text below: \
            Text: {input_text} \
            The output for the identification of the fallacy should be formatted as follows: [{{"text": "", "class": ""}}]. \
            If multiple fallacies occur separate them with commas as follows:  [{{"text": "", "class": ""}}, {{"text": "", "class": ""}}] \
            "text", here you need to quote the sentence in which the fallacy occurs. \
            "class", here you need to give only name of the fallacy that occurs, from the list above, if no fallacy occurs return "no fallacy". \
            '},
    ]
    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

    return outputs[0]["generated_text"][len(prompt):]

## Extract all fallacies for all the data

In [ ]:
import json

file =  open('gold_standard_dataset.jsonl', 'r')
data = [json.loads(line) for line in file]

In [ ]:
results = []

In [ ]:
for n, sample in enumerate(data[0:50]):
  print(n)
  output = gkp_prompt(input_text=sample['text'],
                        pipeline=llama3)
  results.append({'input': sample['text'], 'output': output})

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [ ]:
import csv
keys = results[0].keys()

with open('results.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(results)